In [1]:
import pandas as pd
csv='/mnt/lv/bidur/OkinawaVisitorPred/data/2019-04_2019-08_GyokuSendo.csv'

In [2]:
df = pd.read_csv(csv,usecols=['serial','tripid','tripcount','tlm_datagettime','lat','lon'])
df.rename(columns = {'serial':'ap_id','tlm_datagettime':'timestamp'}, inplace = True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df=df.sort_values(by=['timestamp'])


In [3]:
#df=df.sort_values(by=['ap_id','tripcount','timestamp'])
#df[['ap_id','tripcount','timestamp','lat','lon']].head(20)

In [4]:
arr_ap_ids = df.ap_id.unique()
#ap_id = arr_ap_ids[0]
#ap_id

arr_trip_summary = []

for ap_id in arr_ap_ids:
    df_single_ap = df.query("ap_id=='"+ap_id+"'").copy()
    df_single_ap = df_single_ap.sort_values(by=['timestamp'])
    arr_trips = df_single_ap.tripcount.unique()
    #trip_id = arr_trips[0]
    #trip_id
    ts_prevs = df_single_ap.timestamp.min()
    lat_prevs = df_single_ap.iloc[0].lat
    lon_prevs = df_single_ap.iloc[0].lon
    
    trip_id_prevs = 0
    for trip_id in arr_trips:
        df_single_trip = df_single_ap.query("tripcount=='"+str(trip_id)+"'").copy()
        df_single_trip = df_single_trip.sort_values(by=['timestamp'])
        max_ts = df_single_trip.timestamp.max()
        min_ts= df_single_trip.timestamp.min()
        
        stay_time =  min_ts - ts_prevs
        trip_time = max_ts - min_ts
          
        
        lat_min = df_single_trip.iloc[0].lat
        lon_min = df_single_trip.iloc[0].lon
        lat_max = df_single_trip.iloc[len(df_single_trip)-1].lat 
        lon_max = df_single_trip.iloc[len(df_single_trip)-1].lon
        
        avg_lat = (lat_prevs + lat_min)/2
        avg_lon = (lon_prevs + lon_min)/2
        
        lat_prevs = lat_max
        lon_prevs = lon_max
        
        
        arr_trip_summary.append({
            'ap_id': ap_id,
            
            'trip_prevs': trip_id_prevs,
            'trip_id': trip_id,
            'ts_prevs': ts_prevs,
            'ts_min': min_ts,
          #  'ts_max': max_ts,
          #  'trip_time': trip_time,
            'stay_time': stay_time.total_seconds(),
            'avg_lon_with_prvs': avg_lon,
            'avg_lat_with_prvs': avg_lat,
          #  'lat_min': lat_min,
         #   'lon_min': lon_min,
          #  'lat_max': lat_max,            
          #  'lon_max': lon_max
            })
        trip_id_prevs = trip_id
        ts_prevs = max_ts
    
trip_df =  pd.DataFrame(arr_trip_summary)      
trip_df['date'] = trip_df['ts_min'].dt.date

In [5]:
trip_df['date'] = trip_df['ts_min'].dt.date

In [6]:
trip_df.head(20)

,ap_id,trip_prevs,trip_id,ts_prevs,ts_min,stay_time,avg_lon_with_prvs,avg_lat_with_prvs,date
0,AP521745,0,2080,2019-04-01 08:24:49,2019-04-01 08:24:49,0.0,127.747685,26.142524,2019-04-01
1,AP521745,2080,2081,2019-04-01 08:49:43,2019-04-01 11:14:20,8677.0,127.748598,26.141213,2019-04-01
2,AP521745,2081,2101,2019-04-01 11:16:50,2019-04-04 12:09:51,262381.0,127.747466,26.141419,2019-04-04
3,AP521745,2101,2102,2019-04-04 12:15:45,2019-04-04 18:05:37,20992.0,127.747382,26.141103,2019-04-04
4,AP521745,2102,2123,2019-04-04 18:16:23,2019-04-08 11:20:53,320670.0,127.747085,26.140375,2019-04-08
5,AP521745,2123,2124,2019-04-08 11:23:41,2019-04-08 13:51:25,8864.0,127.749557,26.141103,2019-04-08
6,AP521745,2124,2152,2019-04-08 14:05:35,2019-04-13 10:57:32,420717.0,127.747624,26.140678,2019-04-13
7,AP521745,2152,2153,2019-04-13 10:59:40,2019-04-13 12:58:27,7127.0,127.750141,26.140792,2019-04-13
8,AP521745,2153,2269,2019-04-13 13:00:56,2019-04-25 10:30:56,1027800.0,127.747730,26.142272,2019-04-25
9,AP521745,2269,2270,2019-04-25 10:32:25,2019-04-25 14:24:14,13909.0,127.748954,26.141355,2019-04-25


### remove entries with stay_time < 1800 seconds

In [7]:
trip_df1 = trip_df[trip_df['stay_time']>=1800]

In [8]:
trip_df1.head(20)

,ap_id,trip_prevs,trip_id,ts_prevs,ts_min,stay_time,avg_lon_with_prvs,avg_lat_with_prvs,date
1,AP521745,2080,2081,2019-04-01 08:49:43,2019-04-01 11:14:20,8677.0,127.748598,26.141213,2019-04-01
2,AP521745,2081,2101,2019-04-01 11:16:50,2019-04-04 12:09:51,262381.0,127.747466,26.141419,2019-04-04
3,AP521745,2101,2102,2019-04-04 12:15:45,2019-04-04 18:05:37,20992.0,127.747382,26.141103,2019-04-04
4,AP521745,2102,2123,2019-04-04 18:16:23,2019-04-08 11:20:53,320670.0,127.747085,26.140375,2019-04-08
5,AP521745,2123,2124,2019-04-08 11:23:41,2019-04-08 13:51:25,8864.0,127.749557,26.141103,2019-04-08
6,AP521745,2124,2152,2019-04-08 14:05:35,2019-04-13 10:57:32,420717.0,127.747624,26.140678,2019-04-13
7,AP521745,2152,2153,2019-04-13 10:59:40,2019-04-13 12:58:27,7127.0,127.750141,26.140792,2019-04-13
8,AP521745,2153,2269,2019-04-13 13:00:56,2019-04-25 10:30:56,1027800.0,127.747730,26.142272,2019-04-25
9,AP521745,2269,2270,2019-04-25 10:32:25,2019-04-25 14:24:14,13909.0,127.748954,26.141355,2019-04-25
10,AP521745,2270,2975,2019-04-25 14:28:48,2019-08-09 16:49:45,9166857.0,127.746909,26.141990,2019-08-09
